In [ ]:
from cag.embeddings import SentenceTransformerEmbeddings
from cag.models import ChatOllama
print(1)
embeddings_model = SentenceTransformerEmbeddings('sentence-transformers/all-mpnet-base-v2')
qwen = ChatOllama(model='qwen2.5', temprature=0.001)
llama = ChatOllama(model='llama3.2', temprature=0.001)

# Speed Test of CAG on Intel Core i7 1165G7

In [2]:
import json
import datasets

with open('F:\\OneDrive\\Desktop\\Research\\Dataset\\CRSB-Texts.json', 'r') as f:
    crsb = json.load(f)

crsb = crsb['amazon_rainforest']


squad = datasets.load_dataset('rajpurkar/squad')
squad = squad['validation'].shuffle()
squad = squad[:100]

contexts = crsb['contents']
questions = squad['question']

In [34]:
from cag.vector_candidates.vc import VectorCandidates
from cag.gate.vector_candidates import VectorCandidatesGate

with open('F:\\OneDrive\\Desktop\\Research\\Dataset\\CRSB-Embeddings-MPNET.json', 'r') as f:
    crsb = json.load(f)

crsb_contexts_embeddings = crsb['amazon_rainforest']['contents']
crsb_pseudo_queries_embeddings = crsb['amazon_rainforest']['questions']
VC = VectorCandidates(contexts=[crsb_contexts_embeddings], questions=[crsb_pseudo_queries_embeddings])

Gate = VectorCandidatesGate(vc=VC, embedding_model=embeddings_model)

In [48]:
from time import time
import numpy as np

results = []
acc = []

for i, question in enumerate(questions):
    a = time()
    _, needs_retrieval = Gate(question, policy= 95, threshold= 0.0)
    b = time()
    acc.append(needs_retrieval)
    results.append(b - a)
    

In [49]:
print(f'Mean of the execution : {np.mean(np.array(results))}')

In [50]:
print(acc.count(False))

# Speed Test of Adaptive-RAG on Llama 3.2 on Ollama - Intel Core i7 1165G7

In [51]:
def query_classification(context, query):
    prompt = f"""I will give you a context and a query . please determine if the context is related to the query or not. this means that the context can potentially answer the query or not. 
    first provide the reasons that why the context is related to the query or not; then result a YES or NO answer which determines the context can answer the question or not.
    
    This is the context :
    Context : {context}
    
    Here is the Query :
    Query : {query}
    """
    
    prompt = prompt.format(context = context, query = query)
    
    result = llama.invoke(prompt).content
    
    result = result.lower()
    
    score = True if result.count('yes') > result.count('no') else False
    
    return result, score

In [52]:
import numpy as np
from time import time

adapt_reasons = []
classified = []
times = []

for i, question in enumerate(questions):
    
    a = time()
    x, y = query_classification(contexts[i], question)
    adapt_reasons.append(x)
    classified.append(y)
    b = time()
    
    times.append(b-a)
    
    print(f'Question {i} processed in {b - a} seconds')
    
print(np.mean(np.array(times)))